In [ ]:
#Creating external table
#externbal table must be a directory not file
CREATE EXTERNAL TABLE IF NOT EXISTS dept_big(
  deptno int, 
  dname string, 
  loc string)
COMMENT 'external table with big amount of file'
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\n' 
LOCATION
  'hdfs://nn01.itversity.com:8020/user/rposam2020/output/dept_big';

In [ ]:
#Creating external table
#externbal table must be a directory not file
CREATE EXTERNAL TABLE IF NOT EXISTS emp_big(
  empno int, 
  ename string, 
  job string, 
  mgr int, 
  hiredate string, 
  sal double, 
  comm double, 
  deptno int)
COMMENT 'External table with big amount of file'
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\n'
LOCATION
  'hdfs://nn01.itversity.com:8020/user/rposam2020/output/emp_big';

In [ ]:
#Hive Data Flow
#when query executed -> 
    goes to Hive Driver -> 
        Hive Driver sent query to Compiler
             -> Metastore -> 
                 if valid -> 
                     results back to Driver
    Driver set execution engine ->
        Job Tracker ->
             Map Tasks -> NameNode to get files,blocks,locations -> task done on Datanode
             Reduce Tasks -> NameNode to get files,blocks,locations -> task done on Datanode
        results back to job tracker
    results back to execution enginer
    results back to driver
show results on UI
![image.png](attachment:image.png)

In [ ]:
#Partition and Bucketing concept
#if the source file is big then we split into 4 small files using bucketing
CREATE  TABLE IF NOT EXISTS emp_bucket_part(
  empno int, 
  ename string, 
  job string, 
  mgr int, 
  hiredate string, 
  sal double, 
  comm double)
 COMMENT 'table with big amount of file BUCKET AND PARTITION'
PARTITIONED BY (deptno int)
CLUSTERED BY (JOB) INTO 4 BUCKETS

ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\n'
STORED AS ORC;

In [ ]:
insert into emp_bucket_part partition(deptno) select * from emp_big;

In [ ]:
#Only bucketing
CREATE  TABLE IF NOT EXISTS emp_part(
  empno int, 
  ename string, 
  mgr int, 
  hiredate string, 
  sal double, 
  comm double)
 COMMENT 'table with big amount of file BUCKET AND PARTITION'
PARTITIONED BY (deptno int,job string )
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\n'
STORED AS textfile;

In [ ]:
#if dynamic partition is enable then only this query works other wise static partition inserts has to be done
insert into emp_part partition(deptno,job) 
select empno,ename,mgr,hiredate,sal,comm,deptno,job from emp_big;

#static partition inserting
insert into emp_part partition(deptno='30',job='MANAGER') 
select empno,ename,mgr,hiredate,sal,comm,deptno,job from emp_big
WHERE DEPTNO = 30 AND JOB = 'MANAGER';


In [ ]:
#if you query table using limit it mostly returns rows related to one partition
#so, to get rows randomly from all partitions
select * from emp_part 
order by rand() limit 25;

In [ ]:
#only static partition will be allowed for load
from hdfs
load data inpath "/user/rposam2020/output/emp_big" overwrite into table emp_part;

from local
load data local inpath "/user/rposam2020/data/emp.csv" overwrite into table emp_csv;


#we can use spark2-sql instead of hive
#spark2-sql support all queries of hive and it doesn't process data using map reduce 
#to enable user level privileges we use beeline
!connect jdbc:hive2://rm01.itversity.com:10000
if you don't have access to the other users tables it simply throws an error permission denied
